<a href="https://colab.research.google.com/github/timoz64/DEV6Blog/blob/master/pyspark_mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install pyspark

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('practice').getOrCreate()

In [ ]:
from pyspark.sql.functions import col,lit,create_map

df = spark.createDataFrame([("Alice", 2), ("Bob", 5)], ("name", "age"))
df.select(create_map('name', 'age').alias("map")).collect()

df.select(create_map([df.name, df.age]).alias("map")).collect()


[Row(map={'Alice': 2}), Row(map={'Bob': 5})]

In [ ]:
employee_df = spark.createDataFrame(employee_data, ["ID", "Name", "Age", "Salary"])
filtered_df = employee_df.filter(employee_df.Salary > 5000)
filtered_df.show()

+---+-----+---+------+
| ID| Name|Age|Salary|
+---+-----+---+------+
|  2|Smith| 30|  6000|
|  4|Henry| 40|  7000|
+---+-----+---+------+



In [ ]:
mapped_rdd = employee_rdd.map(lambda x: (x[0], x[1].upper(), x[2], x[3]))
print(mapped_rdd.collect())  # Output: [('1', 'JOHN', 28, 5000), ('2', 'SMITH', 30, 6000), ...]

[('1', 'JOHN', 28, 5000), ('2', 'SMITH', 30, 6000), ('3', 'ADAM', 35, 4000), ('4', 'HENRY', 40, 7000)]


In [ ]:
df1 = spark.createDataFrame(employee_data, ["ID", "Name", "Age", "Salary"])
# df2 = spark.createDataFrame(employee_data, ["ID", "Name", "Age", "Salary"])
df1.show()

+---+-----+---+------+
| ID| Name|Age|Salary|
+---+-----+---+------+
|  1| John| 28|  5000|
|  2|Smith| 30|  6000|
|  3| Adam| 35|  4000|
|  4|Henry| 40|  7000|
+---+-----+---+------+



In [ ]:
# prompt: create dataframe df2, that has only one column "ID" with values 2 and 3

df2 = spark.createDataFrame([(2,), (3,)], ["ID"])
df2.show()

+---+
| ID|
+---+
|  2|
|  3|
+---+



In [ ]:
# add new colum to df1 called new_column that has value 1 for all rows where column ID value equals with column ID value in df2
df1 = df1.join(df2, df1.ID == df2.ID, "left").withColumn("new_column", lit(1))
df1.show()


+---+-----+---+------+----+----------+
| ID| Name|Age|Salary|  ID|new_column|
+---+-----+---+------+----+----------+
|  1| John| 28|  5000|NULL|         1|
|  2|Smith| 30|  6000|   2|         1|
|  3| Adam| 35|  4000|   3|         1|
|  4|Henry| 40|  7000|NULL|         1|
+---+-----+---+------+----+----------+



In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col

# Create the Spark session
spark = SparkSession.builder.getOrCreate()

# Original dataframe
data = [(1, "John", 28, 5000),
        (2, "Smith", 30, 6000),
        (3, "Adam", 35, 4000),
        (4, "Henry", 40, 7000)]
columns = ["ID", "Name", "Age", "Salary"]

df = spark.createDataFrame(data, columns)

# Second dataframe
data2 = [(2,), (3,)]
columns2 = ["ID"]

df2 = spark.createDataFrame(data2, columns2)

# Perform a left join to check for matching IDs
joined_df = df.join(df2, on="ID", how="left")

# Add the new column with the required logic
result_df = joined_df.withColumn(
    "new_column",
    when(col("ID").isin(df2.select("ID").rdd.flatMap(lambda x: x).collect()), 1).otherwise("N/A")
)

# Show the result
result_df.show()


+---+-----+---+------+----------+
| ID| Name|Age|Salary|new_column|
+---+-----+---+------+----------+
|  1| John| 28|  5000|       N/A|
|  2|Smith| 30|  6000|         1|
|  3| Adam| 35|  4000|         1|
|  4|Henry| 40|  7000|       N/A|
+---+-----+---+------+----------+



In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Define the mapping as a dictionary
mapping_dict = {2: "1", 3: "1"}

# Define a UDF for mapping
def map_function(key):
    return mapping_dict.get(key, "N/A")

map_udf = udf(map_function, StringType())

# Add a new column using the UDF
df_with_mapping = df.withColumn("new_column", map_udf(col("ID")))

df_with_mapping.show()


+---+-----+---+------+----------+
| ID| Name|Age|Salary|new_column|
+---+-----+---+------+----------+
|  1| John| 28|  5000|       N/A|
|  2|Smith| 30|  6000|         1|
|  3| Adam| 35|  4000|         1|
|  4|Henry| 40|  7000|       N/A|
+---+-----+---+------+----------+



In [ ]:
from pyspark.sql.functions import lit, when

# Create a mapping dataframe
mapping_df = spark.createDataFrame([(2, 1), (3, 1)], ["ID", "mapped_value"])

# Perform a left join
joined_df = df.join(mapping_df, on="ID", how="left")

# Add a new column with the mapped values or "N/A" if no match
result_df = joined_df.withColumn(
    "new_column",
    when(col("mapped_value").isNotNull(), col("mapped_value")).otherwise("N/A")
).drop("mapped_value")

result_df.show()


+---+-----+---+------+----------+
| ID| Name|Age|Salary|new_column|
+---+-----+---+------+----------+
|  1| John| 28|  5000|       N/A|
|  2|Smith| 30|  6000|         1|
|  3| Adam| 35|  4000|         1|
|  4|Henry| 40|  7000|       N/A|
+---+-----+---+------+----------+



In [ ]:
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.sql.types import *

# Create broadcast dictionary
mapping_dict = {2: "1", 3: "1"}
broadcast_mapping = spark.sparkContext.broadcast(mapping_dict)

# Add a column using a lambda function
df_with_mapping = df.withColumn(
    "new_column",
    F.expr(f"CASE WHEN ID IN ({','.join(map(str, broadcast_mapping.value.keys()))}) THEN 1 ELSE 'N/A' END")
)

df_with_mapping.show()


+---+-----+---+------+----------+
| ID| Name|Age|Salary|new_column|
+---+-----+---+------+----------+
|  1| John| 28|  5000|       N/A|
|  2|Smith| 30|  6000|         1|
|  3| Adam| 35|  4000|         1|
|  4|Henry| 40|  7000|       N/A|
+---+-----+---+------+----------+



In [ ]:
# dataframe to dictionary
# Create a mapping dataframe
mapping_df = spark.createDataFrame([(2, "1"), (3, "1")], ["ID", "mapped_value"])

# Convert the mapping DataFrame to a dictionary
mapping_dict = dict(mapping_df.rdd.map(lambda row: (row["ID"], row["mapped_value"])).collect())

print(mapping_dict)


{2: '1', 3: '1'}
